In [16]:
# Step 1: Install required libraries
!pip install pandas plotly dash --quiet

# Step 2: Import libraries
import pandas as pd
import sqlite3
from dash import Dash, html, dcc
import plotly.express as px
from google.colab import files

# Step 3: Upload your dataset
uploaded = files.upload()
df = pd.read_csv(next(iter(uploaded.keys())))  # Uses your uploaded retail_sales.csv

# Step 4: Create SQLite database
conn = sqlite3.connect(':memory:')
df.to_sql('sales', conn, if_exists='replace', index=False)

# Step 5: SQL Query (Top 10 profitable products)
query = '''
SELECT
    "Product Name",
    SUM("Profit") as total_profit
FROM sales
GROUP BY "Product Name"
ORDER BY total_profit DESC
LIMIT 10
'''
top_products = pd.read_sql(query, conn)

# Step 6: Data Analysis & Visualization
# Monthly Sales Trend
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
monthly_sales = df.groupby(df['Order Date'].dt.to_period('M'))['Sales'].sum().reset_index()
monthly_sales['Order Date'] = monthly_sales['Order Date'].astype(str)

fig1 = px.line(
    monthly_sales,
    x='Order Date',
    y='Sales',
    title='📈 Monthly Sales Trend',
    labels={'Order Date': 'Month', 'Sales': 'Total Sales ($)'}
)

# Product Category Breakdown
fig2 = px.treemap(
    df,
    path=['Category', 'Sub-Category'],
    values='Sales',
    title='🛒 Sales by Product Hierarchy',
    color_discrete_sequence=px.colors.qualitative.Pastel
)

# Step 7: Create Dashboard
app = Dash(__name__)

app.layout = html.Div([
    html.H1("🚀 Retail Sales Dashboard", style={'textAlign': 'center'}),
    dcc.Graph(figure=fig1),
    dcc.Graph(figure=fig2),
    html.H3("💎 Top Profitable Products", style={'textAlign': 'center'}),
    html.Table(
        [html.Tr([html.Th(col) for col in top_products.columns])] +
        [html.Tr([html.Td(top_products.iloc[i][col]) for col in top_products.columns])
         for i in range(len(top_products))],
        style={'margin': 'auto', 'width': '60%', 'textAlign': 'center'}
    )
])

# Step 8: Run the app (updated syntax)
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8050)

Saving retail_sales.csv to retail_sales (5).csv


<IPython.core.display.Javascript object>